# In this nodebook, we are going to cover some of the fundamental concepts of tensors using TensorFlow

More specifically are we are going to cover:

- Introduction to tensors
- Getting information from tensor
- Manipulating Tensors
- Tensors & NumPy
- Using @tf.function (a way to speed up regular Python functions)
- Using GPUs with TensorFlow (or TPUs)
- Exercises for yourself


### Introduction to Tensors


In [2]:
# Import TensorFlow
import tensorflow as tf

print(tf.__version__)

2.15.0


In [4]:
# Create tensors with tf.contants()
scalar = tf.constant(7)
scalar

<tf.Tensor: shape=(), dtype=int32, numpy=7>

In [5]:
# check for number of dimensions of a tensors (ndim stands for number of dimensions)
scalar.ndim

0

In [9]:
# create a vector
vector = tf.constant([10, 10])
vector

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([10, 10], dtype=int32)>

In [10]:
# check dimension of vector
vector.ndim

1

In [12]:
# create a matrix (has more than 1 dimension)
matrix = tf.constant([[10, 7],
                     [7, 10]])
matrix

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 7, 10]], dtype=int32)>

In [13]:
matrix.ndim

2

In [17]:
# create another matrix
another_matrix = tf.constant(
    [
        [4., 2., 7.],
        [3., 8., 10.],
        [6., 5., 9.]
    ],
    dtype=tf.float16)
another_matrix

<tf.Tensor: shape=(3, 3), dtype=float16, numpy=
array([[ 4.,  2.,  7.],
       [ 3.,  8., 10.],
       [ 6.,  5.,  9.]], dtype=float16)>

In [18]:
another_matrix.ndim

2

In [22]:
# let's create a tensor with 3 dimension
tensor = tf.constant(
    [
        [
            [1, 2],
            [3, 4]
        ],

        [
            [5, 6],
            [7, 8]
        ],

        [
            [9, 10],
            [11, 12]
        ]
    ])

tensor

<tf.Tensor: shape=(3, 2, 2), dtype=int32, numpy=
array([[[ 1,  2],
        [ 3,  4]],

       [[ 5,  6],
        [ 7,  8]],

       [[ 9, 10],
        [11, 12]]], dtype=int32)>

In [23]:
tensor.ndim

3

What we have created so far:

- Scalar: A single number
- Vector: A number with direction (e.g. wind speed and direction)
- Matrix: A 2 dimensional array of numbers
- Tensor: A n-dimensional array of numbers (where n can be any number, 0-dimensional is a scalar, 1-dimensional is a vector)


Create tensors with tf.Variable


In [4]:
# create the same tensor with tf.Variable() as above
changeable_tensor = tf.Variable([10, 7])
unchangeable_tensor = tf.constant([10, 7])

changeable_tensor, unchangeable_tensor

(<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([10,  7], dtype=int32)>,
 <tf.Tensor: shape=(2,), dtype=int32, numpy=array([10,  7], dtype=int32)>)

In [6]:
# let's try to change one of the elements in changable tensor
changeable_tensor[0] = 7
changeable_tensor

TypeError: 'ResourceVariable' object does not support item assignment

In [7]:
# How about we try .assign()
changeable_tensor[0].assign(3)
changeable_tensor

<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([3, 7], dtype=int32)>

In [8]:
# Lets try to change unchangable_tensor
unchangeable_tensor[0].assign(3)
unchangeable_tensor

AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'assign'

### Creating Random Tensors

Random tensors are tensors of some arbitary size filled with random numbers


In [5]:
# Create two random (but the same) tensors
random_1 = tf.random.Generator.from_seed(42)  # set seed for reproducability
random_1 = random_1.normal(shape=(3, 2))
random_1

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[-0.7565803 , -0.06854702],
       [ 0.07595026, -1.2573844 ],
       [-0.23193763, -1.8107855 ]], dtype=float32)>

In [6]:
random_2 = tf.random.Generator.from_seed(42)
random_2 = random_2.normal(shape=(3, 2))
random_2

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[-0.7565803 , -0.06854702],
       [ 0.07595026, -1.2573844 ],
       [-0.23193763, -1.8107855 ]], dtype=float32)>

In [7]:
# Are they equal?
random_1, random_2, random_1 == random_2

(<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-0.7565803 , -0.06854702],
        [ 0.07595026, -1.2573844 ],
        [-0.23193763, -1.8107855 ]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-0.7565803 , -0.06854702],
        [ 0.07595026, -1.2573844 ],
        [-0.23193763, -1.8107855 ]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=bool, numpy=
 array([[ True,  True],
        [ True,  True],
        [ True,  True]])>)

### Shuffle the order of elements in tensor


In [15]:
# Shuffle a tensor (valuable when you want to shuffle your data so that, inhirent order does not efffect learning)
not_shuffled = tf.constant(
    [
        [10, 7],
        [3, 4],
        [2, 5]
    ]
)

# shuffle our non_shuffled tensor
shuffled = tf.random.shuffle(not_shuffled)
shuffled

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 2,  5],
       [ 3,  4]], dtype=int32)>

In [58]:
# experementing on seed
tf.random.set_seed(55)  # global level seed
tf.random.shuffle(not_shuffled, seed=55)  # Operational level seed

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[ 2,  5],
       [10,  7],
       [ 3,  4]], dtype=int32)>

### Other ways to make tensors


In [59]:
# Create a tensor of all ones
tf.ones([10, 7])

<tf.Tensor: shape=(10, 7), dtype=float32, numpy=
array([[1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.]], dtype=float32)>

In [60]:
# Create a tensor of all zeros
tf.zeros([3, 4])

<tf.Tensor: shape=(3, 4), dtype=float32, numpy=
array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]], dtype=float32)>

### Turn a numpy arrays to tensors

The main difference between NumPy Array and TensorFlow is that
tensors can be run on a GPU (much faster for numerical operations).


In [63]:
import numpy as np

numpy_A = np.arange(1, 25, dtype=np.int32)  # create a numpy between 1 to 25
numpy_A

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24], dtype=int32)

In [67]:
A = tf.constant(numpy_A)
B = tf.constant(numpy_A, shape=(3, 2, 4))
A, B

(<tf.Tensor: shape=(24,), dtype=int32, numpy=
 array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24], dtype=int32)>,
 <tf.Tensor: shape=(3, 2, 4), dtype=int32, numpy=
 array([[[ 1,  2,  3,  4],
         [ 5,  6,  7,  8]],
 
        [[ 9, 10, 11, 12],
         [13, 14, 15, 16]],
 
        [[17, 18, 19, 20],
         [21, 22, 23, 24]]], dtype=int32)>)

In [68]:
3 * 2 * 4

24

### Getting Information from Tensors

When dealing with Tensors are probably wanted to be aware of following attributes:

- Shape
- Rank
- Axis or Dimension
- Size


In [5]:
# Create a Rank 4 Tensor (4 dimensions)
rank_4_tensor = tf.zeros(shape=[2, 3, 4, 5])
rank_4_tensor

<tf.Tensor: shape=(2, 3, 4, 5), dtype=float32, numpy=
array([[[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]],


       [[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]]], dtype=float32)>

In [6]:
rank_4_tensor[0]

<tf.Tensor: shape=(3, 4, 5), dtype=float32, numpy=
array([[[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]]], dtype=float32)>

In [7]:
rank_4_tensor.shape, rank_4_tensor.ndim, tf.size(rank_4_tensor)

(TensorShape([2, 3, 4, 5]), 4, <tf.Tensor: shape=(), dtype=int32, numpy=120>)

In [8]:
2 * 3 * 4 * 5

120

In [18]:
# Get various attributes of our tensor
print(f"DataType of every element: {rank_4_tensor.dtype}")
print(f"Number of dimensions (rank): {rank_4_tensor.ndim}")
print(f"Shape of Tensor: {rank_4_tensor.shape}")
print(f"Elements along 0 axis: {rank_4_tensor.shape[0]}")
print(f"Elements along last axis: {rank_4_tensor.shape[-1]}")
print(f"Total number of Elements in Tensor: {tf.size(rank_4_tensor)}")

DataType of every element: <dtype: 'float32'>
Number of dimensions (rank): 4
Shape of Tensor: (2, 3, 4, 5)
Elements along 0 axis: 2
Elements along last axis: 5
Total number of Elements in Tensor: 120


### Indexing Tensors

Tensors can be indexed just like Python list


In [21]:
some_list= [1,2,3,4]
some_list[:2]

[1, 2]

In [20]:
# Get the first elements of each dimension
rank_4_tensor[:2, :2, :2, :2]

<tf.Tensor: shape=(2, 2, 2, 2), dtype=float32, numpy=
array([[[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]],


       [[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]]], dtype=float32)>

In [24]:
# Get the first elemtent from each dimension from each index except the final one
rank_4_tensor[:1, :1, :1, :]

<tf.Tensor: shape=(1, 1, 1, 5), dtype=float32, numpy=array([[[[0., 0., 0., 0., 0.]]]], dtype=float32)>

In [26]:
# Create a rank 2 tensor (2 dimension)
rank_2_tensor = tf.constant(
    [
        [10, 7],
        [4, 5]
    ]
)
rank_2_tensor.shape, rank_2_tensor.ndim

(TensorShape([2, 2]), 2)

In [34]:
# Get the last element of each of our rank 2 tensor
rank_2_tensor[:,-1]

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([7, 5], dtype=int32)>

In [39]:
# Add a new dimension to our rank 2 tensor
rank_3_tensor = rank_2_tensor[..., tf.newaxis]
rank_3_tensor

<tf.Tensor: shape=(2, 2, 1), dtype=int32, numpy=
array([[[10],
        [ 7]],

       [[ 4],
        [ 5]]], dtype=int32)>

In [40]:
# Alternative of tf.newaxis
tf.expand_dims(rank_2_tensor, axis=-1) # '-1' means expand the final axis

<tf.Tensor: shape=(2, 2, 1), dtype=int32, numpy=
array([[[10],
        [ 7]],

       [[ 4],
        [ 5]]], dtype=int32)>

In [41]:
# expand 0-axis
tf.expand_dims(rank_2_tensor, axis=0)  # '0' means expand the 0 axis

<tf.Tensor: shape=(1, 2, 2), dtype=int32, numpy=
array([[[10,  7],
        [ 4,  5]]], dtype=int32)>